
# Two-photon Functional Preprocessing (L395_f10)

This notebook loads repository metadata, derives the preprocessing parameters for a two-photon functional session, and runs the preprocessing pipeline. Output artefacts mirror the existing `02_reg/00_preprocessing/2p_functional` layout, but everything is written into `D:/pipelineTestOutput` for testing.

> Set `REPROCESS = True` if you need to overwrite existing outputs.


In [ ]:
from pathlib import Path
import logging

from social_imaging_scripts.metadata.loader import load_animals
from social_imaging_scripts.preprocessing.two_photon import functional, motion
from social_imaging_scripts.metadata.config import load_project_config
from social_imaging_scripts.metadata.config import resolve_raw_path


logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")

In [ ]:

cfg = load_project_config()
PROJECT_ROOT = Path.cwd().parent
ANIMAL_ID = "L395_f10"
OUTPUT_BASE = cfg.output_base_dir  # resolves to /mnt/f/johannes/testoutput on WSL, D:/social_imaging_outputs on Windows

METADATA_BASE = PROJECT_ROOT / "metadata/animals"

FAST_DISK = OUTPUT_BASE  # Path('D:/pipelineTestOutput/temp_suite2p') if you want binaries elsewhere
OPS_PATH = Path("../suite2p_ops_may2025.npy")

REFERENCE_BRAIN_RELATIVE = Path("03_Common_Use/reference brains/ref_05_LB_Perrino_2p/average_2p_noRot_orig.nrrd")
reference_brain_path = resolve_raw_path(REFERENCE_BRAIN_RELATIVE, cfg=cfg)

In [ ]:
REPROCESS = False # whether to re-run plane splitting if outputs already exist
MOTION_REPROCESS = False
FPS = 2.0  # Hz


ops_template = motion.load_global_ops(OPS_PATH)
ops_template['nonrigid'] = True

In [ ]:

collection = load_animals(base_dir=METADATA_BASE)
animal = collection.by_id(ANIMAL_ID)

functional_sessions = [s for s in animal.sessions if s.session_type == "functional_stack"]
if not functional_sessions:
    raise RuntimeError(f"No functional sessions found for {ANIMAL_ID}")

session = functional_sessions[0]
settings = session.session_data.preprocessing_two_photon
if settings is None:
    raise RuntimeError("Two-photon preprocessing settings missing in metadata")

raw_dir = Path(session.session_data.raw_path)
output_root = OUTPUT_BASE / ANIMAL_ID / "02_reg" / "00_preprocessing" / "2p_functional"

print("Raw directory:", raw_dir)
print("Output root:", output_root)
print("Mode:", settings.mode)
print("Planes:", settings.n_planes)
print("Frames per plane:", settings.frames_per_plane)
print("Blocks:", settings.blocks)


## FireANTs anatomy registration

Run the new FireANTs pipeline (winsorize → affine → greedy) to align the two-photon anatomy stack to the Perrino reference brain.

In [ ]:
from pathlib import Path

from social_imaging_scripts.registration import (
    FireANTsRegistrationConfig,
    register_two_photon_anatomy,
)
from social_imaging_scripts.metadata.config import resolve_raw_path, resolve_output_path

FIREANTS_CONFIG = FireANTsRegistrationConfig()
print(FIREANTS_CONFIG)

In [ ]:
# Resolve inputs/outputs based on metadata
anatomy_sessions = [
    s
    for s in animal.sessions
    if s.session_type == "anatomy_stack"
    and getattr(s.session_data, "stack_type", "") == "two_photon"
]
if not anatomy_sessions:
    raise RuntimeError(f"No two-photon anatomy session found for {ANIMAL_ID}")

anatomy_session = anatomy_sessions[0]
raw_anatomy = Path(animal.root_dir) / anatomy_session.session_data.raw_path
raw_anatomy_full = resolve_raw_path(raw_anatomy, cfg=cfg)
if not raw_anatomy_full.exists():
    raise FileNotFoundError(raw_anatomy_full)

reference_path = resolve_raw_path(
    Path("03_Common_Use/reference brains/ref_05_LB_Perrino_2p/average_2p_noRot_orig.nrrd"),
    cfg=cfg,
)
if not reference_path.exists():
    raise FileNotFoundError(reference_path)

registration_output_root = resolve_output_path(
    ANIMAL_ID,
    "02_reg",
    "02_fireants",
    cfg=cfg,
)
registration_output_root.mkdir(parents=True, exist_ok=True)

print("Moving stack:", raw_anatomy_full)
print("Reference brain:", reference_path)
print("Output root:", registration_output_root)

In [ ]:
fireants_results = register_two_photon_anatomy(
    animal_id=ANIMAL_ID,
    session_id=anatomy_session.session_id,
    stack_path=raw_anatomy_full,
    reference_path=reference_path,
    output_root=registration_output_root,
    config=FIREANTS_CONFIG,
)

print("FireANTs outputs:")
for key, value in fireants_results.items():
    print(f"  {key}: {value}")

In [ ]:
from IPython.display import display, Image

qc_path = fireants_results.get("qc")
if qc_path and Path(qc_path).exists():
    display(Image(filename=qc_path))
else:
    print("QC file not found")